In [1]:
import os
import requests
import time
import re
from bs4 import BeautifulSoup
import m3u8
import sys
from tqdm import tqdm

search_url = "https://ww.9anime2.com/search/"
watch_url = "https://ww.9anime2.com/watch/"
base = "https://ww.9anime2.com"

In [2]:
anime = input("Enter Anime Name : ")
search_term = "%20".join(anime.split())

Enter Anime Name : naruto


In [3]:
url = search_url+search_term
r = requests.get(url)
soup = BeautifulSoup(r.text, features="lxml")

In [4]:
titles = soup.find("ul", {"class": "anime-list"})
anime_list = titles.find_all("li")
title_list = ["\t".join(i.text.replace("\ndub","").replace(" Series","").replace("Special ","Special").split("\n")[3:]) for i in anime_list]

In [5]:
print()
print(str(len(title_list)) + " Results found matching name : " + anime)
print("\nIndex \t Type \t Name\n")
for i in range(len(title_list)):
    print(str(i+1) + "\t " + title_list[i])


40 Results found matching name : naruto

Index 	 Type 	 Name

1	 TV 	 Naruto 
2	 Movie 	 Naruto Shippuuden Movie 1 
3	 Movie 	 Naruto Shippuuden Movie 7: The Last 
4	 Movie 	 Boruto: Naruto the Movie (Dub) 
5	 Movie 	 Boruto: Naruto the Movie 
6	 TV 	 Boruto: Naruto Next Generations (Dub) 
7	 Spring 2017 Anime 	 Boruto: Naruto Next Generations 
8	 Movie 	 Naruto: Shippuuden Movie 6 - Road to Ninja (Dub) 
9	 Movie 	 Naruto: Shippuuden Movie 1 (Dub) 
10	 Movie 	 Naruto Shippuuden Movie 2: Kizuna 
11	 TV 	 Naruto Shippuden 
12	 TV 	 Naruto (Dub) 
13	 Movie 	 Boruto: Naruto the Movie (Dub) 
14	 Special	 Boruto: Naruto the Movie - Naruto ga Hokage ni Natta Hi 
15	 Movie 	 Naruto: Shippuuden Movie 2 - Kizuna (Dub) 
16	 Movie 	 The Last: Naruto the Movie (Dub) 
17	 Movie 	 Naruto Shippuuden Movie 6: Road to Ninja 
18	 Movie 	 Naruto: Shippuuden Movie 5 - Blood Prison (Dub) 
19	 Special	 Naruto: Akaki Yotsuba no Clover wo Sagase 
20	 Movie 	 Naruto Shippuuden Movie 5: Blood Prison 
21	 TV 	 N

In [6]:
print()
choice = input("Enter anime index to download : ")

suffix = "-".join(title_list[int(choice)-1].split('\t')[1].split()).lower()
url = "https://ww.9anime2.com/watch/"+suffix
r = requests.get(url)
soup = BeautifulSoup(r.text, features="lxml")


Enter anime index to download : 1


In [7]:
episodes = soup.find("ul", {"class": "episodes"})
noe = len(episodes.find_all('li'))

In [8]:
print('\n-------------------------\n')
print("Number of downloadable episodes in the series : " + str(noe))


-------------------------

Number of downloadable episodes in the series : 220


In [9]:
choice = input("\nEnter episode(s) to be downloaded (space-separatred): ")
choice = re.sub("[^0-9]", " ", choice)


Enter episode(s) to be downloaded (space-separatred): 1 2 3 4 5 6 7 8 9


In [10]:
choices = choice.split()
for i in choices:
    if(int(i) not in range(1,noe+1)):
        print("Latest episode count is "+str(noe)+". ")
        print("Skipping episode "+i)
        choices.remove(i)
        print()

In [11]:
choices = list(sorted(set(choices)))

In [12]:
DOWNLOAD_LINKS = []
print('\n-------------------------\n')
print("Collecting download links \n")


-------------------------




In [13]:
for i in tqdm(choices):
    URL = url+"/"+i
    episode_page = requests.get(URL)
    soup = BeautifulSoup(episode_page.text, features="lxml")
    embed = soup.find("iframe", {"id": "playerframe"}).get('src')
    embed_url = base + embed
    embed_page = requests.get(embed_url)
    soup = BeautifulSoup(embed_page.text, features="lxml")
    data  = str(soup.find_all("script")[-1].string)
    link = re.search("(?P<url>https?://[^\s]+)", data).group("url").replace("'","")
    if link[-10:] == 'index.m3u8':
        link = link[:-10]
    DOWNLOAD_LINKS.append(link)
    


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:11<00:00,  1.27s/it]


In [14]:
def download(link, filename):
    m3u8_file = link +"index.m3u8"
    m3u8_data = requests.get(m3u8_file)
    m3u8_master = m3u8.loads(m3u8_data.text)
    uris = [link + x['uri'] for x in m3u8_master.data['segments']]
    sess = requests.Session()
    with open(filename + ".ts", 'wb') as f:
        for uri in tqdm(uris):
            r = sess.get(uri)
            f.write(r.content)

In [15]:
print('\n-------------------------\n')
print("Initiating Download \n")


-------------------------

Initiating Download 



In [17]:
# for i in range(len(DOWNLOAD_LINKS)):
#     print("Downloading ep "+choices[i])
#     download(DOWNLOAD_LINKS[i], suffix + "_episode_"+choices[i])

In [18]:
print("\nDOWNLOAD FINISHED !! ")


DOWNLOAD FINISHED !! 


In [ ]:
input()